# Preprocessing Code for Twitter Dataset

In [1]:
#!pip install emoji

import pandas as pd
import numpy as np
import regex as re
import string
import nltk
import emoji

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

stop_words = set(stopwords.words('english')) - {'all'}

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/edvardbruun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edvardbruun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/edvardbruun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/edvardbruun/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Gets the part of speech tag of word for lemmatization
# This function is based on code from:
#   https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

# Preprocesses the tweets text
# This function is based on code from:
#   https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python
def preprocess_text(tweet):
    # Changes emojis to words
    tweet = emoji.demojize(tweet,  delimiters=(' ', ' '))
    # Removes 'RT' from tweet
    tweet = re.sub(r'RT[\s]+', '', tweet)
    # Removes capitalization
    tweet = tweet.lower()
    # Removes urls & user mentions from tweet
    tweet = re.sub(r"http\S+|www\S+|https\S+|\@\w+", ' ', tweet, flags=re.MULTILINE)
    # Removes punctuation
    tweet = re.sub(r'\p{P}+', '', tweet)
    # Removes stopwords
    tokens = [w for w in word_tokenize(tweet) if not w in stop_words]
    # Perfoms lemmatization on tokens
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    return " ".join(lemma_words)

# Preprocesses the text of the Tweets in the df and returns the df
# By default, this removes the Tweets with the "neither" label
def preprocess_df(df, remove_neither=True):
  idx = "text"
  length = len(df[idx])
  for ii in range(length):
    tweet = str(df[idx][ii])
    df.loc[ii, idx] = preprocess_text(tweet)
  if (remove_neither):
    return df[df['BLM'] != "neither"]
  else:
    return df

In [23]:
# Retrieves and preprocesses the training dataset
path = "./train.csv" # Path to train.csv
train_df = pd.read_csv(path)
train_df.fillna("", inplace=True) # fills any NaN values with empty strings
train_df = preprocess_df(train_df)
train_df.head(5)

TypeError: unsupported operand type(s) for +: 'method' and 'int'

In [48]:
# Uses a CountVectorizer to construct bag-of-words matrix
vectorizer = CountVectorizer() # Add a comment about the max_features & ngram_range parameters

# train_vocab is an 2d array of the vocab from the training dataset 
train_vocab = vectorizer.fit_transform(train_df['text']).toarray()

# train_vocab_df is a dataframe where the element ij is the number of times word j occurred in Tweet i
train_vocab_df = pd.DataFrame(train_vocab, columns=vectorizer.get_feature_names())
train_labels = train_df['BLM']

In [37]:
train_df.head()
#print (train_labels.count() + 1)

#train_vocab_df.head()
#print (train_vocab_df.count() + 1)

#train_labels.head()
#print (train_labels.count() + 1)

,created_at,hashtags,text,BLM
0,2013-08-05,BlackLivesMatter BrownLivesMatter Every28Hours...,let talk state violence youth color blacklives...,positive
2,2013-08-30,blacklivesmatter,mt show kid positive image black people build ...,positive
3,2013-08-30,BlackLivesMatter BBW13,q1 big parent influence blacklivesmatter bbw13,positive
4,2013-08-30,BlackLivesMatter,a10 breastfeeding life love crucial beautiful ...,positive
5,2013-08-30,BlackLivesMatter BBW13,new people jumping cohosting blacklivesmatter ...,positive


In [13]:
# Retrieves and preprocesses the test dataset
path = "./test.csv" # Path to Test_dataset.csv
test_df = pd.read_csv(path)
test_df.fillna("", inplace=True) # fills any NaN values with empty strings
test_df = preprocess_df(test_df)
test_df.head(5)

,created_at,hashtags,text,BLM
0,2013-08-30,BlackLivesMatter,a3 big challenge find balance ambitious want p...,positive
1,2014-08-02,RememberTynirah Amnesty2014,remembertynirah all girl deserve chance all li...,positive
2,2014-08-15,Ferguson NMOS14 BlackLivesMatter Atlanta,national moment silence atlanta ferguson nmos1...,positive
3,2014-08-17,lgbtq Ferguson blacklivesmatter outrage racism,hey big shout lgbtq solidarity ferguson blackl...,positive
4,2014-08-18,BlackLivesMatter PeaceInFerguson,share sunday sermon tenacious nonconformist fa...,positive


In [47]:
# Uses the vocab from the training dataset to vectorize the test dataset
test_vocab = vectorizer.transform(test_df['text']).toarray()

# test_vocab_df is a dataframe where the element ij is the number of times word j
# occurred in Tweet i
test_vocab_df = pd.DataFrame(test_vocab, columns=vectorizer.get_feature_names())
test_labels = test_df['BLM']


10614
10614


In [19]:
test_vocab_df.head()
test_labels.head()

0    positive
1    positive
2    positive
3    positive
4    positive
Name: BLM, dtype: object

In [38]:
print(f"Number of neither Tweets in training: {len(train_df[train_df['BLM'] == 'neither'])}")
print(f"Number of positive Tweets in training: {len(train_df[train_df['BLM'] == 'positive'])}")
print(f"Number of negative Tweets in training: {len(train_df[train_df['BLM'] == 'negative'])}")

Number of neither Tweets in training: 0
Number of positive Tweets in training: 5528
Number of negative Tweets in training: 1219


In [39]:
print(f"Number of neither Tweets in test: {len(test_df[test_df['BLM'] == 'neither'])}")
print(f"Number of positive Tweets in test: {len(test_df[test_df['BLM'] == 'positive'])}")
print(f"Number of negative Tweets in test: {len(test_df[test_df['BLM'] == 'negative'])}")

Number of neither Tweets in test: 0
Number of positive Tweets in test: 1383
Number of negative Tweets in test: 305
